In [3]:
#check the direction

In [4]:
import os
print(os.getcwd())

C:\Program Files\Microsoft VS Code


In [5]:
import os

# Change to your project directory (update the path as needed)
os.chdir(r"H:/Work_with_Mis_Ataran")

# Verify the change
print(os.getcwd())

H:\Work_with_Mis_Ataran


In [6]:
import pandas as pd

# Read the Excel file
Data = pd.read_excel("data_validation\Jaghargh.xlsx")
# Show the first 5 rows
print(Data.head())

        Date   Hour  Hourly_Rain
0 2022-12-04  09:00          0.1
1 2023-02-20  10:00          0.1
2 2023-02-22  12:00          0.0
3 2023-02-27  12:00          0.0
4 2023-02-27  14:00          0.1


In [7]:
Data

,Date,Hour,Hourly_Rain
0,2022-12-04,09:00,0.1
1,2023-02-20,10:00,0.1
2,2023-02-22,12:00,0.0
3,2023-02-27,12:00,0.0
4,2023-02-27,14:00,0.1
...,...,...,...
3599,2025-04-02,12:00,0.0
3600,2025-04-02,18:00,0.0
3601,2025-04-02,23:00,0.0
3602,2025-04-03,06:00,0.0


In [10]:
import xarray as xr

# Specify the path to your NetCDF file
file_path = r"Pr2024.nc"

# Open the NetCDF file
p = xr.open_dataset(file_path)

In [12]:
p

<xarray.Dataset> Size: 74MB
Dimensions:                     (time: 8784, lon: 50, lat: 42)
Coordinates:
  * time                        (time) datetime64[ns] 70kB 2024-01-01 ... 202...
  * lon                         (lon) float64 400B 56.28 56.38 ... 61.08 61.18
  * lat                         (lat) float64 336B 33.51 33.61 ... 37.51 37.61
Data variables:
    total_precipitation_hourly  (time, lon, lat) float32 74MB ...
Attributes: (12/33)
    date_range:               [ 347155200000 1640649600000]
    description:              <p>ERA5-Land is a reanalysis dataset providing ...
    keywords:                 ['cds', 'climate', 'copernicus', 'ecmwf', 'era5...
    period:                   0
    product_tags:             ['temperature', 'lakes', 'snow', 'soil_water', ...
    provider:                 Climate Data Store
    ...                       ...
    visualization_3_bands:    u_component_of_wind_10m
    visualization_3_max:      30.0
    visualization_3_min:      0.0
    visualization_3_name:     u-component of wind at 10m
    visualization_3_palette:  #FFFFFF,#FFFF71,#DEFF00,#9EFF00,#77B038,#007E55...
    crs:                      EPSG:4326

In [20]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import regionmask
import os

# Load the NetCDF file
file_path = r"Pr1990_2024_combined.nc"
era5_ds = xr.open_dataset(file_path)
precip = era5_ds['total_precipitation_hourly']

# Convert precipitation from meters to millimeters
precip_mm = precip * 1000  # 1 m = 1000 mm

# Load the shapefile
shapefile_path = r"Province_Khorasan/province.shp"
gdf = gpd.read_file(shapefile_path)

# Reproject shapefile to WGS84 if necessary
if gdf.crs != 'EPSG:4326':
    gdf = gdf.to_crs(epsg=4326)

# Create a mask using explicit coordinates
mask = regionmask.from_geopandas(gdf).mask(precip_mm.lon, precip_mm.lat)

# Apply the mask to the precipitation data
precip_clipped = precip_mm.where(mask >= 0)

# Compute mean, max, and median over the time dimension
mean_precip = precip_clipped.mean(dim='time')
max_precip = precip_clipped.max(dim='time')
median_precip = precip_clipped.median(dim='time')

# Create output directory for plots
output_dir = "clipped_precipitation_maps"
os.makedirs(output_dir, exist_ok=True)

# Define the map projection and set extent based on ERA5 data
projection = ccrs.PlateCarree()
extent = [era5_ds.lon.min().values - 1, era5_ds.lon.max().values + 1,
          era5_ds.lat.min().values - 1, era5_ds.lat.max().values + 1]

# Function to plot a map
def plot_map(data, title, filename, cmap='Blues', vmin=None, vmax=None):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    # Plot the clipped data
    im = data.plot(ax=ax, transform=ccrs.PlateCarree(), cmap=cmap, vmin=vmin, vmax=vmax, 
                   cbar_kwargs={'label': 'Precipitation (mm)'})
    
    # Add shapefile boundaries
    gdf.boundary.plot(ax=ax, edgecolor='black', linewidth=1.5, transform=ccrs.PlateCarree())
    
    # Add map features
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.LAKES, edgecolor='black')
    
    # Set title
    ax.set_title(title)
    
    # Save the plot
    plot_path = os.path.join(output_dir, filename)
    plt.savefig(plot_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"Map saved at {plot_path}")

# Plot maps for mean, max, and median
plot_map(mean_precip, 'Mean Hourly Precipitation (2024) - Khorasan Province', 
         'mean_precip_map_clipped.png', cmap='Blues')
plot_map(max_precip, 'Maximum Hourly Precipitation (2024) - Khorasan Province', 
         'max_precip_map_clipped.png', cmap='Reds')
plot_map(median_precip, 'Median Hourly Precipitation (2024) - Khorasan Province', 
         'median_precip_map_clipped.png', cmap='Greens')

# Close the NetCDF file
era5_ds.close()

Map saved at clipped_precipitation_maps\mean_precip_map_clipped.png
Map saved at clipped_precipitation_maps\max_precip_map_clipped.png
Map saved at clipped_precipitation_maps\median_precip_map_clipped.png


In [14]:
p['total_precipitation_hourly'].max()

<xarray.DataArray 'total_precipitation_hourly' ()> Size: 4B
array(0.01135898, dtype=float32)

In [17]:
import xarray as xr
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import os
from datetime import datetime

# Define station information with paths
stations = [
    {"name": "Jaghargh", "lat": 36.3114864, "lon": 59.3224514, "path": "data_validation/Jaghargh.xlsx"},
    {"name": "Zoshk", "lat": 36.3385960, "lon": 59.1953066, "path": "data_validation/Zoshk.xlsx"},
    {"name": "Edareh", "lat": 36.3217200, "lon": 59.5684300, "path": "data_validation/Edareh.xlsx"},
    {"name": "Radkan", "lat": 36.8039817, "lon": 59.0101517, "path": "data_validation/Radkan.xlsx"},
    {"name": "Andarkh", "lat": 36.5820760, "lon": 59.6607562, "path": "data_validation/Andarkh.xlsx"},
    {"name": "Chenaran", "lat": 36.6462594, "lon": 59.1161501, "path": "data_validation/Chenaran.xlsx"},
    {"name": "Shandiz", "lat": 36.3995647, "lon": 59.3389961, "path": "data_validation/Shandiz.xlsx"},
    {"name": "Ferizi", "lat": 36.4884911, "lon": 58.9787934, "path": "data_validation/Ferizi.xlsx"},
    {"name": "Torogh", "lat": 36.1727771, "lon": 59.5526764, "path": "data_validation/Torogh.xlsx"}
]

# Load ERA5 NetCDF file
file_path = r"Pr1990_2024_combined.nc"
era5_ds = xr.open_dataset(file_path)
era5_precip = era5_ds['total_precipitation_hourly']

# Convert ERA5 precipitation from meters to millimeters
era5_precip_mm = era5_precip * 1000  # 1 m = 1000 mm

# Function to find nearest grid point
def find_nearest_grid(lat, lon, lat_array, lon_array):
    lat_idx = np.abs(lat_array - lat).argmin()
    lon_idx = np.abs(lon_array - lon).argmin()
    return lat_idx, lon_idx

# Create output directory for plots
output_dir = "scatter_plots"
os.makedirs(output_dir, exist_ok=True)

# Process each station
for station in stations:
    name = station['name']
    lat = station['lat']
    lon = station['lon']
    excel_path = station['path']
    
    # Load observation data
    try:
        obs_df = pd.read_excel(excel_path)
    except FileNotFoundError:
        print(f"Excel file for {name} not found at {excel_path}. Skipping...")
        continue
    
    # Combine Date and Hour into a single datetime column
    obs_df['DateTime'] = pd.to_datetime(obs_df['Date'].astype(str) + ' ' + obs_df['Hour'].astype(str))
    obs_df.set_index('DateTime', inplace=True)
    
    # Extract ERA5 data for the nearest grid point
    lat_idx, lon_idx = find_nearest_grid(lat, lon, era5_ds.lat.values, era5_ds.lon.values)
    era5_station = era5_precip_mm.isel(lat=lat_idx, lon=lon_idx).to_dataframe()['total_precipitation_hourly']
    
    # Align ERA5 data with observation dates
    common_dates = obs_df.index.intersection(era5_station.index)
    if len(common_dates) == 0:
        print(f"No overlapping dates for {name}. Skipping...")
        continue
    
    obs_data = obs_df.loc[common_dates, 'Hourly_Rain']
    era5_data = era5_station.loc[common_dates]
    
    # Remove NaN values for correlation and plotting
    valid_mask = ~obs_data.isna() & ~era5_data.isna()
    obs_valid = obs_data[valid_mask]
    era5_valid = era5_data[valid_mask]
    
    # Calculate Pearson correlation
    if len(obs_valid) >= 2:
        corr, p_value = pearsonr(obs_valid, era5_valid)
        corr_text = f'Correlation: {corr:.3f}\np-value: {p_value:.3e}'
        print(f"{name} Correlation: {corr:.3f}, p-value: {p_value:.3e}")
    else:
        corr_text = 'Insufficient data for correlation'
        print(f"Insufficient valid data for correlation at {name}.")
    
    # Plotting scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(obs_valid, era5_valid, alpha=0.5, color='blue')
    # Add 1:1 line
    max_val = max(obs_valid.max(), era5_valid.max())
    plt.plot([0, max_val], [0, max_val], 'r--', lw=2, label='1:1 Line')
    plt.xlabel('Observation Precipitation (mm)')
    plt.ylabel('ERA5 Precipitation (mm)')
    plt.title(f'ERA5 vs Observation Precipitation at {name} (ERA5 in mm)')
    plt.legend()
    plt.grid(True)
    
    # Add correlation text to the plot
    plt.text(0.02, 0.98, corr_text, transform=plt.gca().transAxes, 
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    
    # Save plot
    plot_path = os.path.join(output_dir, f"{name}_scatter.png")
    plt.savefig(plot_path)
    plt.close()
    print(f"Scatter plot saved for {name} at {plot_path}")

# Close the NetCDF file
era5_ds.close()

Jaghargh Correlation: 0.358, p-value: 7.792e-94
Scatter plot saved for Jaghargh at scatter_plots\Jaghargh_scatter.png
Zoshk Correlation: 0.324, p-value: 6.865e-153
Scatter plot saved for Zoshk at scatter_plots\Zoshk_scatter.png
Edareh Correlation: 0.262, p-value: 2.106e-61
Scatter plot saved for Edareh at scatter_plots\Edareh_scatter.png
Radkan Correlation: 0.344, p-value: 0.000e+00
Scatter plot saved for Radkan at scatter_plots\Radkan_scatter.png
Andarkh Correlation: 0.337, p-value: 0.000e+00
Scatter plot saved for Andarkh at scatter_plots\Andarkh_scatter.png
Chenaran Correlation: 0.368, p-value: 1.213e-122
Scatter plot saved for Chenaran at scatter_plots\Chenaran_scatter.png
Shandiz Correlation: 0.166, p-value: 1.266e-132
Scatter plot saved for Shandiz at scatter_plots\Shandiz_scatter.png
Ferizi Correlation: 0.267, p-value: 0.000e+00
Scatter plot saved for Ferizi at scatter_plots\Ferizi_scatter.png
Torogh Correlation: -0.013, p-value: 7.817e-01
Scatter plot saved for Torogh at scatte